In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name ='/home/arno/Projects/EEGDecodingTest/My/LLM/Qwen2.5-7B-Instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt} 
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
# model_inputs['inputs_embeds']=2233
input_embeds=model.get_input_embeddings()(model_inputs['input_ids'])




/home/arno/conda/envs/MNE-EEG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  6.00it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name ='/home/arno/Projects/EEGDecodingTest/My/LLM/Qwen2.5-7B-Instruct'
device="cuda"

class ModifiedQwenModel(AutoModelForCausalLM):
    def __init__(self, config):
        super().__init__(config)
    
    def forward(self, input_ids=None, attention_mask=None, input_embeds=None, **kwargs):
        # 获取模型的 transformer
        transformer = self.transformer
        
        print(input_ids.shape)
        print(input_embeds.shape)
        
        if input_embeds is None:
            hidden_states = transformer.wte(input_ids)  # 输入嵌入（Embedding）
        else:
            hidden_states = input_embeds
            
        # return hidden_states

        # Apply the embedding dropout (if any)
        if transformer.dropout is not None:
            hidden_states = transformer.dropout(hidden_states)

        # Apply position encodings
        hidden_states = hidden_states + transformer.wpe(torch.arange(input_ids.size(1), device=input_ids.device))  # Position Embeddings

        # 通过 transformer 的第2个 block 开始
        for i in range( len(transformer.h)):  # 跳过第一个 block
            hidden_states = transformer.h[i](hidden_states, attention_mask=attention_mask)

        # 计算 logits
        logits = transformer.lm_head(hidden_states)
        return logits
    
    
model = ModifiedQwenModel.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/home/arno/conda/envs/MNE-EEG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.89it/s]


In [9]:
prompt = "日本这个国家怎么样"

messages = [
            # {"role": "system", "content": ' You are a helpful assistant. The user has a language impairment, and their expressions may contain a lot of noise. You need to rephrase their meaning clearly.'},
            {"role": "system", "content": ' You are a helpful assistant.'},
            {"role": "user", "content": prompt}
            ]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# text=prompt

model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512, do_sample=True)

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

system
 You are a helpful assistant.
user
日本这个国家怎么样
assistant
日本是一个位于东亚的岛国，拥有悠久的历史和文化。以下是一些关于日本的基本情况：

1. 文化：日本文化深受中国、韩国以及西方的影响。它有许多独特的传统艺术形式，如茶道、花道、歌舞伎等。此外，动漫、流行音乐和电子游戏也是日本文化的重要组成部分。

2. 经济：日本是世界第三大经济体（按购买力平价计算），以高科技产业和制造业著称。日本企业在汽车制造、电子设备等领域具有很强的竞争力。

3. 科技：日本在科技创新方面处于领先地位，在机器人技术、电动汽车、可再生能源等方面取得了显著成就。

4. 教育：日本重视教育，国民受教育程度较高。日本大学在全球排名中占有一定位置。

5. 自然风光：日本拥有美丽的自然景观，包括富士山、温泉、樱花盛开的季节等。

6. 交通：日本拥有发达的公共交通系统，包括高速铁路、地铁和公交等。

7. 社会问题：尽管日本经济繁荣，但面临一些社会问题，如人口老龄化、少子化等。

总之，日本是一个充满魅力的国家，具有丰富的文化和历史遗产，同时也面临着一些挑战。


In [22]:
prompt = "日本这个国家怎么样"

messages = [
            # {"role": "system", "content": ' You are a helpful assistant. The user has a language impairment, and their expressions may contain a lot of noise. You need to rephrase their meaning clearly.'},
            {"role": "system", "content": ' You are a helpful assistant.'},
            {"role": "user", "content": prompt}
            ]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# text=prompt

model_inputs = tokenizer([text], return_tensors="pt").to(device)
predicted_ids=model.generate(inputs_embeds=model.get_input_embeddings()(model_inputs.input_ids) ,
                             max_new_tokens=512, do_sample=True)
response = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(response)


system
 You are a helpful assistant.
user
日本这个国家怎么样
assistant
日本是一个位于东亚的岛国，拥有悠久的历史和文化。以下是一些关于日本的基本情况：

1. 文化：日本文化深受中国、韩国以及西方的影响。它有许多独特的传统艺术形式，如茶道、花道、歌舞伎等。此外，动漫、流行音乐和电子游戏也是日本文化的重要组成部分。

2. 经济：日本是世界第三大经济体（按购买力平价计算），以高科技产业和制造业著称。日本企业在汽车制造、电子设备等领域具有很强的竞争力。

3. 科技：日本在科技创新方面处于领先地位，在机器人技术、电动汽车、可再生能源等方面取得了显著成就。

4. 教育：日本重视教育，国民受教育程度较高。日本大学在全球排名中占有一定位置。

5. 自然风光：日本拥有美丽的自然景观，包括富士山、温泉、樱花盛开的季节等。

6. 交通：日本拥有发达的公共交通系统，包括高速铁路、地铁和公交等。

7. 社会问题：尽管日本经济繁荣，但面临一些社会问题，如人口老龄化、少子化等。

总之，日本是一个充满魅力的国家，具有丰富的文化和历史遗产，同时也面临着一些挑战。


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name ='/home/arno/Projects/EEGDecodingTest/My/LLM/Qwen2.5-7B-Instruct'
device="cuda"
    
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def getEmbedding(text, raw=False):
        if not raw:
                messages = [
                        # {"role": "system", "content": ' You are a helpful assistant. The user has a language impairment, and their expressions may contain a lot of noise. You need to rephrase their meaning clearly.'},
                        {"role": "system", "content": ' You are a helpful assistant.'},
                        {"role": "user", "content": text}
                        ]
                text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        return model.get_input_embeddings()(model_inputs.input_ids)

prompt = '“天…天，天气...嗯，外面，嗯，亮，阳光，太阳，感觉…暖。嗯，风，风有点，嗯，吹，嗯，冷，是，不是...不太...不太冷。啊，今天，今天是，嗯…可以，出门，出去，走，走，没问题。嗯，暖，舒服。”'

# messages = [
#             {"role": "system", "content": ' You are a helpful assistant. The user has a language impairment, and their expressions may contain a lot of noise. You need to rephrase their meaning clearly.'},
#             {"role": "user", "content": prompt}
#             ]
# text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# model_inputs = tokenizer([text], return_tensors="pt").to(device)
# generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512, do_sample=True)




new_input_embed = torch.cat([getEmbedding('<|im_start|>system\nYou are a helpful assistant. The user has a language impairment, and their expressions may contain a lot of noise. You need to rephrase their meaning clearly.<|im_end|>\n<|im_start|>user\n', raw=True)
                             , getEmbedding(prompt, raw=True), 
                             getEmbedding('<|im_end|>\n<|im_start|>assistant\n', raw=True)], dim=1)

predicted_ids=model.generate(inputs_embeds=new_input_embed ,
                             max_new_tokens=512, do_sample=True)



response = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(response)


/home/arno/conda/envs/MNE-EEG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.90it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


今天的天气感觉挺暖和的。外面阳光明媚，虽然有风，但不算冷。今天出门应该没问题，挺舒服的。


In [58]:
import pickle

tokenids=[]
texts=[]
with open('/home/arno/Projects/EEGDecodingTest/My/Data/qwen-characterSplit/sub04/run_1_with_latent_tokenid.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
    for  _, text, latent, tokenid in loaded_data:
        tokenids.append(tokenid[0])
        texts.append(text)
len(tokenids)

3479

In [44]:
characterEmbeddings=model.get_input_embeddings()(torch.tensor(tokenids).to(device=device))

In [70]:
character_embeddings_flat

tensor([ 0.0004, -0.0034,  0.0013,  ..., -0.0013,  0.0033,  0.0123],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<ViewBackward0>)

In [81]:
import torch

# 假设 characterEmbeddings 是一个 PyTorch tensor，形状是 [3479, 3584]
# 先展平数据以计算95%的置信区间
character_embeddings_flat = characterEmbeddings.view(-1).to(torch.float32) 

# 计算95%置信区间
lower_bound = torch.quantile(character_embeddings_flat, 0.025)
upper_bound = torch.quantile(character_embeddings_flat, 0.975)

# 计算新的标准差 std_new，基于95%的置信区间
std_new = ((upper_bound - lower_bound) / (2 * 1.96)).cpu().item()

# 获取均值
mean_old = torch.mean(character_embeddings_flat).cpu().item()


# 生成与原数据形状相同的噪声
noise = torch.normal(mean=mean_old, std=std_new, size=characterEmbeddings.shape)

# 生成带噪声的数据
noisy_data = characterEmbeddings + noise.to(device=device)

# 打印一下结果的均值和标准差，确认噪声分布
print(f"Mean of noisy data: {torch.mean(noisy_data).item()}")
print(f"Std of noisy data: {torch.std(noisy_data).item()}")


Mean of noisy data: 6.160988414194435e-05
Std of noisy data: 0.022888336330652237


In [82]:
noisy_data.to(torch.bfloat16)

torch.Size([3479, 3584])

In [84]:
new_input_embed = torch.cat([getEmbedding('<|im_start|>system\nYou are a helpful assistant. The user has a language impairment, and their expressions may contain a lot of noise. You need to rephrase their meaning clearly.<|im_end|>\n<|im_start|>user\n', raw=True)
                             , noisy_data.to(torch.bfloat16).unsqueeze(0), 
                             getEmbedding('<|im_end|>\n<|im_start|>assistant\n', raw=True)], dim=1)

predicted_ids=model.generate(inputs_embeds=new_input_embed ,
                             max_new_tokens=512, do_sample=True)



response = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(response)

1. 在六岁那年，我读过一本描绘原始森林的书，并在书中看到了一幅精彩的插图。图上画着一条蟒蛇吞吃一头猛兽的情景。书上写到蟒蛇会将猎物囫囵吞下，不咀嚼，然后躺上六个月来消化它们。当时我对丛林里的奇妙景象很感兴趣，于是用彩色铅笔画下了我的第一幅画——“作品一号”。我把这幅杰作给大人看，问他们这幅画是不是很吓人。他们回答说：“一顶帽子怎么可能会吓人呢？”我说：“我画的不是一顶帽子，而是一条蟒蛇在消化大象。”因此，我又画了蟒蛇肚子里的大象内部结构，以便让大人看得更清楚。他们总是要求我解释我的“作品二号”，但那些大人劝我不要再画蟒蛇，不管是剖开的还是没剖开的，都不要画了。他们说应该把心思放在地理、历史、算术和语法上。当时我才六岁，就放弃了辉煌的画家生涯。“作品一号”和“作品二号”都没成功，我感到泄气。那些大人自己也弄不懂我的作品，总是要孩子一遍遍地解释，这让我很烦。后来我选择了学习飞行，学会了开飞机，几乎飞遍了世界各地。地理学对我很有帮助，我可以一眼认出中国在哪里，亚利桑那在哪里。如果晚上迷路了，也很有用。我在那些大人中间生活了很久，仔细观察过他们。我发现他们似乎并没有什么特别之处。如果遇到一个头脑稍微清醒一点的人，我就拿出一直保存的“作品一号”，试着让他看看。我想知道他是否真的能看懂。但人们总回答说：“这是顶帽子。”这时，我就不再提蟒蛇、原始森林或星星之类的事情，而是说些他们能理解的事。比如桥、高尔夫、政治和领带等。

2. 我孤独地生活着，没有真正谈得来的朋友。直到六年前，有一天飞机出了故障降落在撒哈拉沙漠。发动机里有东西坏了，因为身边既没有机械师也没有乘客，我打算独自完成一项困难的修复工作。这是一个生死攸关的问题。我带的水只够喝一个星期。第一天晚上，我睡在这片远离人烟的沙漠中，比在大海中漂浮的遇难者还孤独。当黎明时分，有一个奇怪的声音


In [61]:
print(*texts)

1 我 六 岁 那 年 在 一 本 描 写 原 始 森 林 的 名 叫 真 实 的 故 事 的 书 上 看 见 过 一 幅 精 彩 的 插 图 画 的 是 一 条 蟒 蛇 在 吞 吃 一 头 猛 兽 我 现 在 把 它 照 样 画 在 上 面 书 中 写 道 蟒 蛇 把 猎 物 囫 囵 吞 下 嚼 都 不 嚼 然 后 它 就 无 法 动 弹 躺 上 六 个 月 来 消 化 它 们 当 时 我 对 丛 林 里 的 奇 妙 景 象 想 得 很 多 于 是 我 也 用 彩 色 铅 笔 画 了 我 的 第 一 幅 画 我 的 作 品 一 号 它 就 像 这 样 我 把 这 幅 杰 作 给 大 人 看 问 他 们 我 的 图 画 吓 不 吓 人 他 们 回 答 说 一 顶 帽 子 怎 么 会 吓 人 呢 我 画 的 不 是 一 顶 帽 子 我 画 的 是 一 条 蟒 蛇 在 消 化 大 象 于 是 我 把 蟒 蛇 肚 子 的 内 部 画 出 来 好 让 这 些 大 人 看 得 明 白 他 们 老 是 要 人 给 他 们 解 释 我 的 作 品 二 号 是 这 样 的 那 些 大 人 劝 我 别 再 画 蟒 蛇 甭 管 它 是 剖 开 的 还 是 没 剖 开 的 全 都 丢 开 他 们 说 我 还 是 把 心 思 放 在 地 理 历 史 算 术 和 语 法 上 好 就 这 样 我 才 六 岁 就 放 弃 了 辉 煌 的 画 家 生 涯 作 品 一 号 和 作 品 二 号 都 没 成 功 我 泄 了 气 那 些 大 人 自 个 儿 什 么 也 弄 不 懂 老 要 孩 子 们 一 遍 一 遍 给 他 们 解 释 真 烦 人 我 只 好 另 外 选 择 一 个 职 业 学 会 了 开 飞 机 世 界 各 地 我 差 不 多 都 飞 过 的 确 地 理 学 对 我 非 常 有 用 我 一 眼 就 能 认 出 哪 是 中 国 哪 是 亚 利 桑 那 要 是 夜 里 迷 了 路 这 很 有 用 就 这 样 我 这 一 生 中 跟 好 多 严 肃 的 人 打 过 好 多 交 道 我 在 那 些 大 人 中 间 生 活 过 很 长 时 间 我 仔 细 地 观 察 过 他 们 观 察 下 来 印 象 并 没 好 多 少 要 是 碰 上 一 个 人 看 上 去 头 脑 稍 许 清 楚 些 我 就 